# Italian Names Scraper (from Wikipedia)

## Import Libraries

In [1]:
from ntscraper import Nitter
import requests
import pprint
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import matplotlib.pyplot as plt
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Define Scraper Function

In [40]:
def load_names_form_wiki(url, n_pages = 2, button_text = 'next page'):
    # Create a new instance of the Chrome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("useAutomationExtension", False)
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    driver = webdriver.Chrome(options=chrome_options)
    
    # Open the URL in the browser
    driver.get(url)
    
    # Wait for the "Load More" button to be present
    wait = WebDriverWait(driver, 10)
    load_more_button = wait.until(EC.presence_of_element_located((By.LINK_TEXT, button_text)))
    
    html_content = driver.page_source
    
    iteration = 0
    while load_more_button and iteration <= n_pages:
        try:
            print(f"Iteration {iteration}/{n_pages}: Loading more names...")
            # Click the "Load More" button
            load_more_button.click()
            # Wait for new tweets to load
            # wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.loading')))
            time.sleep(2)
            iteration += 1
            if driver.find_elements(By.LINK_TEXT, 'No more content to load'):
                break
            # Check if there's another "Load More" button
            load_more_button = driver.find_elements(By.LINK_TEXT, button_text)[0] if driver.find_elements(By.LINK_TEXT, button_text) else False
            html_content += driver.page_source
        except KeyboardInterrupt:
            print("Process interrupted by the user")
            break
        except:
            print("Some error occured")
            break
    # Get the updated page source after loading all tweets
    # html_content = driver.page_source
    
    # Close the browser
    driver.quit()
    
    return html_content

## Scrape Male Names

In [11]:
names_html = load_names_form_wiki('https://en.wikipedia.org/w/index.php?title=Category:Italian_masculine_given_names&pageuntil=Giuliano#mw-pages')

Iteration 0/2: Loading more names...


In [12]:
soup = BeautifulSoup(names_html, 'html.parser')

In [34]:
all_names = soup.find_all('div', class_ ="mw-category mw-category-columns")
names_1 = [name.text for name in all_names[0].find_all('a')]
names_1 = [name.replace('(given name)', '') for name in names_1]
names_1 = [name.replace('(name)', '') for name in names_1]
names_1 = [name.replace(' ', '') for name in names_1]
names_2 = [name.text for name in all_names[1].find_all('a')]
names_2 = [name.replace('(given name)', '') for name in names_2]
names_2 = [name.replace('(name)', '') for name in names_2]
names_2 = [name.replace(' ', '') for name in names_2]
male_names = names_1 + names_2
print(len(male_names))

399


## Scrape Female Names

In [36]:
url = 'https://it.wikipedia.org/w/index.php?title=Categoria:Prenomi_italiani_femminili&pageuntil=Elba+%28nome%29#mw-pages'

In [43]:
names_html = load_names_form_wiki(url, 4, 'pagina successiva')
soup = BeautifulSoup(names_html, 'html.parser')
all_names = soup.find_all('div', class_ ="mw-category mw-category-columns")
names_1 = [name.text for name in all_names[0].find_all('a')]
names_1 = [name.replace('(given name)', '') for name in names_1]
names_1 = [name.replace('(name)', '') for name in names_1]
names_1 = [name.replace(' ', '') for name in names_1]
names_2 = [name.text for name in all_names[1].find_all('a')]
names_2 = [name.replace('(given name)', '') for name in names_2]
names_2 = [name.replace('(name)', '') for name in names_2]
names_2 = [name.replace(' ', '') for name in names_2]
names_3 = [name.text for name in all_names[2].find_all('a')]
names_3 = [name.replace('(given name)', '') for name in names_3]
names_3 = [name.replace('(name)', '') for name in names_3]
names_3 = [name.replace(' ', '') for name in names_3]
names_4 = [name.text for name in all_names[3].find_all('a')]
names_4 = [name.replace('(given name)', '') for name in names_4]
names_4 = [name.replace('(name)', '') for name in names_4]
names_4 = [name.replace(' ', '') for name in names_4]
female_names = names_1 + names_2 + names_3 + names_4
female_names = [name.replace('(nome)', '') for name in female_names]
print(len(female_names))

Iteration 0/4: Loading more names...
Iteration 1/4: Loading more names...
Iteration 2/4: Loading more names...
626


In [46]:
female_names[-10:]

['Wanda',
 'Wilma',
 'Xenia',
 'Yvonne',
 'Zaira',
 'Zelmira',
 'Zenaide',
 'Zina',
 'Zita',
 'Zoe']

## Write txt

In [47]:
with open('male_names.txt', 'w') as f:
    for name in male_names:
        f.write(name + '\n')
with open('female_names.txt', 'w') as f:
    for name in female_names:
        f.write(name + '\n')